<a href="https://colab.research.google.com/github/busiri/busil/blob/main/%EC%9E%AC%EB%AC%B4%EC%A7%80%ED%91%9C%ED%86%B5%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data\파생위한재무종합.csv",index_col=0)
df['회계년도'] = df['회계년도'].str.split('/').str[0]
df['회계년도'] = df['회계년도'].astype('int64')
df.drop(columns=['연결_추가여부', '개별_추가여부', 'kos_gaap_추가여부','비상장_연결_추가여부', '비상장_개별_추가여부', '비상장_gaap_추가여부'],inplace=True)

In [ ]:
df1 = pd.read_csv('data\이자비용처리 전 데이터.csv',index_col=0)
join_data = df1.merge(df,how='left',on=['회사명','거래소코드','회계년도'])

In [ ]:
join_data = join_data[join_data['이자보상배율(이자비용)'] != 999999999]

In [ ]:
join_data['GAAP'] = join_data['kos_gaap_추가여부'] + join_data['out_gaap_1_추가여부'] + join_data['out_gaap_2_추가여부'] + \
join_data['out_gaap_3_추가여부'] + join_data['out_gaap_4_추가여부']
join_data.rename(columns={'연결_추가여부' : 'IFRS_CONN','개별_추가여부' : 'IFRS'},inplace=True)

In [ ]:
# 0) 로그 변환 지표 추가
join_data['/ 로그매출액'] = np.log(join_data['매출액(천원)'].replace(0, np.nan))
join_data['/ 로그자산']   = np.log(join_data['자산(천원)'].replace(0, np.nan))
join_data['EBIT'] = join_data['법인세비용차감전손익(천원)'] + join_data['이자비용(천원)']

c:\Users\fursew\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\fursew\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
join_data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
def add_derived_vars_no_lag(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 0) 로그 변환 지표 추가
    df['/ 로그매출액'] = np.log(df['매출액(천원)'].replace(0, np.nan))
    df['/ 로그자산']   = np.log(df['자산(천원)'].replace(0, np.nan))

    # 안전 나눗셈 도우미
    def safe_div(num, denom):
        return np.where(denom == 0, np.nan, num / denom)

    # 1) 안정성 비율
    df['/ 현금/총자산 비율'] = safe_div(df['현금및현금성자산(천원)'], df['자산(천원)'])
    df['/ 이익잉여금/총자산 비율'] = safe_div(df['이익잉여금(천원)'], df['자산(천원)'])
    df['/ 급여/총자산 비율'] = safe_div(df['급여(천원)'],df['자산(천원)'])

    # 2) 비용·레버리지·효율
    df['/ 급여/매출액 비율'] = safe_div(df['급여(천원)'],df['매출액(천원)'])
    df['/ 업력 효율지수'] = safe_div(df['매출액(천원)'],df['업력'])
    df['/ 매출원가/부채 비율'] = safe_div(df['매출원가(천원)'],df['부채(천원)'])
    df['/ 급여/세전순익 비율'] = safe_div(df['급여(천원)'],df['법인세비용차감전손익(천원)'])
    df['/ 영업 레버리지'] = safe_div(df['영업이익증가율'],df['매출액증가율'])
    df['/ 재무 레버리지'] = safe_div(
        df['영업이익(손실)(천원)'],
        df['영업이익(손실)(천원)'] - df['이자비용(천원)']
    )
    df['/ 결합 레버리지'] = df['/ 영업 레버리지'] * df['/ 재무 레버리지']

    # 3) 추가 비용·레버리지 지표
    df['/ 금융비용부담률'] = safe_div(df['이자비용(천원)'], df['매출액(천원)'])

    # 4) 현금흐름 변수
    df['/ 영업CF/자산 비율'] = safe_div(df['영업현금흐름(천원)'],df['자산(천원)'])
    df['/ 투자CF/자산 비율'] = safe_div(df['투자활동으로 인한 현금흐름(*)(천원)'], df['자산(천원)'])
    df['/ 영업CF/유동부채 비율'] = safe_div(df['영업현금흐름(천원)'],df['유동부채(천원)'])
    total_borrow = df.get('총차입금(천원)', df['부채(천원)'])
    df['/ 재무CF/총차입금 비율'] = safe_div(df['재무활동으로 인한 현금흐름(*)(천원)'], total_borrow)

    # 5) 통합현금흐름 긍정여부
    cf_sum = (
        df['영업현금흐름(천원)']
      + df['투자활동으로 인한 현금흐름(*)(천원)']
      + df['재무활동으로 인한 현금흐름(*)(천원)']
    )
    df['/ 현금흐름통합_긍정여부'] = np.where(cf_sum > 0, 1, 0)

    # 6) 투하자본수익률 (영업이익 ÷ (자본+부채) ×100)
    total_cap = df['자본(천원)'] + df['부채(천원)']
    df['/ 투하자본수익률'] = safe_div(df['영업이익(손실)(천원)'], total_cap) * 100

    # 7) 무한대 → NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

In [ ]:
final = add_derived_vars_no_lag(join_data)

c:\Users\fursew\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\fursew\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
final['EBIT'] = final['법인세비용차감전손익(천원)'] + final['이자비용(천원)']
final['EBIT / 자산'] = final['EBIT']/final['자산(천원)']
final['EBIT / 매출액'] = final['EBIT']/final['매출액(천원)']
final['EBIT / 영업현금흐름(천원)'] = final['EBIT']/final['영업현금흐름(천원)']

In [ ]:
final.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
flag = join_data['업력'] >= 0
join_data = join_data.loc[flag]

In [ ]:
drop_cols = ['kos_gaap_추가여부',
 'out_gaap_1_추가여부',
 'out_gaap_2_추가여부',
 'out_gaap_3_추가여부',
 'out_gaap_4_추가여부']
join_data.drop(columns=drop_cols,inplace=True)

In [ ]:
eda_data = join_data[join_data['회계년도']>2011]

In [ ]:
eda_data.columns.to_list()

['회사명',
 '거래소코드',
 '회계년도',
 '총자본증가율',
 '유형자산증가율',
 '비유동자산증가율',
 '유동자산증가율',
 '재고자산증가율',
 '자기자본증가율',
 '매출액증가율',
 '영업이익증가율',
 '순이익증가율',
 '종업원1인당 부가가치증가율',
 '종업원수증가율',
 '종업원1인당 매출액증가율',
 '종업원1인당 인건비증가율',
 '매출액총이익률',
 '영업이익률(매출액)',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본영업이익률',
 '총자본순이익률',
 '자기자본영업이익률',
 '자기자본순이익률',
 '경영자본영업이익률',
 '경영자본순이익률',
 '자본금영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '영업비용대영업수익비율',
 '금융비용부담률',
 '외환이익 대 매출액비율',
 '광고선전비 대 매출액비율',
 '세전이익대비세금율(차감전)',
 '기업순이익률',
 '수지비율',
 '인건비 대 총비용비율',
 'R & D 투자효율',
 '총비용대비세금율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '사내유보율',
 '사내유보 대 자기자본비율',
 '적립금비율(재정비율)',
 '평균배당률',
 '자기자본배당률',
 '배당성향',
 '1주당매출액(원)',
 '유동자산구성비율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '당좌자산구성비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '자기자본배율',
 '비유동비율',
 '비유동장기적합률',
 '유동비율',
 '당좌비율',
 '현금비율',
 '매출채권비율',
 '재고자산 대 순운전자본비율',
 '매출채권 대 매입채무비율',
 '매출채권 대 상,제품비율',
 '매입채무 대 재고자산비율',
 '부채비율',
 '유동부채비율',
 '단기차입금 대 총차입금비율',
 '비유동부채

In [ ]:
거시 = ['미국실업률변화율',
 '유로실업률변화율',
 '일본실업률변화율',
 '영국실업률변화율',
 '무역수지변화율',
 '미국소비자물가지수변화율',
 '소비자판매액지수변화율',
 '실질GDP변화율',
 '연준금리변화율',
 'GDP변화율',
 '소비자심리지수변화율',
 '미국비농업고용지수변화율',
 '정책금리변화율',
 '외국인투자동향(전체)변화율',
 'Lf 평잔변화율',
 '경제심리지수변화율',
 '수입물량지수변화율',
 '수출물량지수변화율',
 '수입물가지수변화율',
 '수출물가지수변화율',
 '소비자물가지수변화율',
 '경기종합지수변화율',
 '경상수지변화율',
 '생산자물가지수변화율',
 '예금은행 총수신변화율',
 '외환보유액변화율',
 'spread_10y_3y변화율',
 'spread_10y_3m변화율',
 'credit_spread_AA변화율',
 'credit_spread_BBB변화율',
 '나스닥변화율',
 '유로화변화율',
 '파운드화변화율',
 '엔화변화율',
 'LNG변화율',
 'WTI유변화율',
 '가솔린변화율',
 '구리변화율',
 '난방유변화율',
 '납변화율',
 '니켈변화율',
 '대두변화율',
 '면변화율',
 '밀변화율',
 '설탕변화율',
 '쌀변화율',
 '알루미늄변화율',
 '옥수수변화율',
 '주석변화율',
 '천연가스변화율',
 '커피변화율',
 '코코아변화율',
 '금 수익률',
 '돈육 수익률',
 '백금 수익률',
 '생우 수익률',
 '육우 수익률',
 '은 수익률',
 '명목 달러 인덱스 변화율',
 '실질 달러 인덱스 변화율',
 'VIX변화율',
 'US_spread_10y_3y변화율',
 'US_spread_10y_3m변화율',
 'US_credit_spread_AA변화율',
 'credit_spread_BBB.1변화율',
 'DOW JONES변화율',
 'NASDAQ변화율',
 'S&P500변화율',
 'kospi변화율',
 'kosdaq변화율',
 'G20경기선행지수변화율',
 'Korea경기선행지수변화율',
 'US경기선행지수변화율',
 'China경기선행지수변화율']
eda_not_market = eda_data.drop(columns=거시)
eda_not_market.shape

(229940, 164)

In [ ]:
eda_data.drop(columns=[ '이자보상배율(이자비용)',
 '이자보상배율(순금융비용)',
'이자보상배율(영업이익)'
],inplace=True)

C:\Users\fursew\AppData\Local\Temp\ipykernel_14412\1827388257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eda_data.drop(columns=[ '이자보상배율(이자비용)',


In [ ]:
object_cols = [
'IFRS_CONN',
 'IFRS',
'대표이사변경여부',
 '수도권','부실여부','GAAP']

for col in object_cols:
    eda_data[col] = eda_data[col].astype('object')

In [ ]:
eda = eda_not_market.select_dtypes(include=[np.number])

In [ ]:
join_data['업력_범주11'] = join_data['업력'].map(lambda x : 1 if x>11 else 0)
join_data['업력_범주12'] = join_data['업력'].map(lambda x : 1 if x>12 else 0)
join_data['업력_범주15'] = join_data['업력'].map(lambda x : 1 if x>15 else 0)

In [ ]:
final.drop(columns=drop_cols,inplace=True)

In [ ]:
final_data = df[df['회계년도']>2011]

In [ ]:
final_data.shape

(302105, 27)

In [ ]:
final_data.to_csv('data/EDA데이터.csv',index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('data/국면나누기전.csv')
df.head()

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,순이익증가율,...,/ 재무CF/총차입금 비율,/ 현금흐름통합_긍정여부,/ 투하자본수익률,EBIT / 자산,EBIT / 매출액,EBIT / 영업현금흐름(천원),업력_범주11,업력_범주12,업력_범주15,생애주기
0,(유)가든주류,97082,2019,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.093048,1,5.990455,0.078677,0.051786,18.886403,1,1,1,성장기
1,(유)가든주류,97082,2020,-0.92,11.15,17.83,-10.63,-29.66,-40.28,-70.11,...,0.034579,0,1.868142,0.032636,0.024388,-3.145047,1,1,1,도입기
2,(유)가든주류,97082,2021,-10.44,-54.68,-33.54,5.33,-40.32,8.31,45.89,...,-0.196926,0,0.462901,0.046543,0.036091,10.508698,1,1,1,기타
3,(유)가든주류,97082,2022,-66.57,-10.70,-49.37,-73.98,-8.32,0.00,0.00,...,0.375515,1,-27.880289,-0.266720,-0.080142,0.099535,1,1,1,쇠퇴기
4,(유)그룹세브코리아,13171,2019,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.481217,0,23.705408,0.239545,0.065874,0.853058,1,1,1,성숙기


In [ ]:
import pandas as pd
def year_check(x: pd.Series) -> int:
    return int(len(x) != (x.max() - x.min() + 1))

df['회계년도연속여부'] = df.groupby(['회사명', '거래소코드'])['회계년도'].transform(
    lambda x: year_check(x)
)

In [ ]:
df['회계년도연속여부'].value_counts()

회계년도연속여부
0    205141
1     24799
Name: count, dtype: int64

In [ ]:
df[['회사명','회계년도','회계년도연속여부']]

,회사명,회계년도,회계년도연속여부
0,(유)가든주류,2019,0
1,(유)가든주류,2020,0
2,(유)가든주류,2021,0
3,(유)가든주류,2022,0
4,(유)그룹세브코리아,2019,0
...,...,...,...
229935,힐티코리아주식회사,2019,0
229936,힐티코리아주식회사,2020,0
229937,힐티코리아주식회사,2021,0
229938,힐티코리아주식회사,2022,0


### 데이터 분할

In [ ]:
### 국면으로 나누기
op_col = '영업현금흐름(천원)'
inv_col = '투자활동으로 인한 현금흐름(*)(천원)'
fin_col = '재무활동으로 인한 현금흐름(*)(천원)'

# 국면 분류 함수 정의
def classify_dickinson(row):
    op = row[op_col]
    inv = row[inv_col]
    fin = row[fin_col]

    if op < 0 and inv < 0 and fin > 0:
        return '도입기'
    elif op > 0 and inv < 0 and fin > 0:
        return '성장기'
    elif op > 0 and inv < 0 and fin < 0:
        return '성숙기'
    elif op < 0 and inv > 0:
        return '쇠퇴기'
    else:
        return '기타'

# 데이터프레임에 생애주기 컬럼 추가
eda['생애주기'] = eda.apply(classify_dickinson, axis=1)

# 원하는 순서 정의
stage_order = ['도입기', '성장기', '성숙기', '쇠퇴기', '기타']

# value_counts 후 원하는 순서로 정렬
counts = eda['생애주기'].value_counts().loc[stage_order]

# 출력
print(counts)
print("\n총 기업 수:", counts.sum())

filtered_lifecycle = eda[eda['생애주기'] != '기타'].copy()

df_도입 = filtered_lifecycle[filtered_lifecycle['생애주기']=='도입기'].drop('생애주기',axis=1)
df_성장 = filtered_lifecycle[filtered_lifecycle['생애주기']=='성장기'].drop('생애주기',axis=1)
df_성숙 = filtered_lifecycle[filtered_lifecycle['생애주기']=='성숙기'].drop('생애주기',axis=1)
df_쇠퇴 = filtered_lifecycle[filtered_lifecycle['생애주기']=='쇠퇴기'].drop('생애주기',axis=1)

생애주기
도입기    42314
성장기    53975
성숙기    68220
쇠퇴기    23977
기타     41454
Name: count, dtype: int64

총 기업 수: 229940


In [ ]:
df_도입.to_csv('data/eda/eda도입.csv',index=False)
df_성장.to_csv('data/eda/eda성장.csv',index=False)
df_성숙.to_csv('data/eda/eda성숙.csv',index=False)
df_쇠퇴.to_csv('data/eda/eda쇠퇴.csv',index=False)

In [ ]:
df_도입['상장폐지일']

1         0
9         0
12        0
13        0
15        0
         ..
229927    0
229928    0
229930    0
229932    0
229933    0
Name: 상장폐지일, Length: 42314, dtype: object

In [ ]:
df_list = [df_도입,df_성장,df_성숙,df_쇠퇴]

for df in df_list:
    for col in object_cols:
        df[col] = df[col].astype('object')

In [ ]:
df_도입.to_csv('도입기.csv',index=False)
df_성장.to_csv('성장기.csv',index=False)
df_성숙.to_csv('성숙기.csv',index=False)
df_쇠퇴.to_csv('쇠퇴기.csv',index=False)